In [ ]:
from PIL import Image
import pandas as pd
import jsonlines
import os



### Create Directories

In [ ]:
import os 

parent_directory = os.path.dirname(os.path.dirname(os.getcwd()))
data_directory = os.path.join(parent_directory,'data')

# Open the JSONL file in read mode using jsonlines
with jsonlines.open(os.path.join(data_directory,'dev.jsonl')) as reader:
    # Iterate over each line in the file
    for data in reader:
       
        print(data)

### Load in Transformer model from HuggingFace

##### VisualBert and Tokenizer

In [ ]:
import torch 
from transformers import BertTokenizer, VisualBertModel

model = VisualBertModel.from_pretrained('uclanlp/visualbert-vqa-coco-pre')
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

In [ ]:
model.config

##### Swinv2Model for image processing

In [ ]:
from transformers import Swinv2Model,AutoImageProcessor

viz_model = Swinv2Model.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")#.from_pretrained('yusx-swapp/ofm-swinv2-base-patch4-window7-cifar100')
image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")#.from_pretrained('yusx-swapp/ofm-swinv2-base-patch4-window7-cifar100')

##### Create DataFrame from train.jsonl

In [ ]:
import os 

parent_directory = os.path.dirname(os.path.dirname(os.getcwd()))
data_directory = os.path.join(parent_directory,'data')

train_dataset = pd.DataFrame(columns=['id','img','label','text'])

# Open the JSONL file in read mode using jsonlines
with jsonlines.open(os.path.join(data_directory,'train.jsonl')) as reader:
    # Iterate over each line in the file
    for data in reader:
        # Process the data as needed
        data_df = pd.DataFrame([data])
        train_dataset = pd.concat([train_dataset,data_df]).reset_index(drop=True)


#### Swinv2 vs. ViT

In [ ]:
from transformers import AutoImageProcessor, Swinv2Model
import torch
from datasets import load_dataset
import numpy as np
from PIL import Image

image = Image.open(os.path.join(data_directory,train_dataset.loc[0,'img']))
image_array = np.array(image)
tensor = torch.tensor(image_array)


image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")
viz_model = Swinv2Model.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")

inputs = image_processor(tensor, return_tensors="pt")

with torch.no_grad():
    outputs = viz_model(**inputs)

visual_embeds = outputs.last_hidden_state
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)

In [ ]:
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
feature_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

inputs = feature_extractor(images=tensor, return_tensors="pt")
outputs = feature_model(**inputs)#.to('cuda'))

visual_embeds = outputs['last_hidden_state']
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)


#### Create input dataset for VisualBERT by connecting all the pieces

In [ ]:
train_dataset

In [ ]:
train_dataset.loc[0,'text']

In [ ]:
input_text = train_dataset.loc[0,'text']
input_text_tokenized = tokenizer(input_text, return_tensors='pt', padding='max_length', max_length=512, truncation=True)



In [ ]:
input_text_tokenized.update(
    {
        "visual_embeds": visual_embeds,
        "visual_token_type_ids": visual_token_type_ids,
        "visual_attention_mask": visual_attention_mask,
    }
)

### Components of VisualBERT input

      1. Text input 
      2. Image input

    1. Text input requires three pieces: input_ids, attention_mask, token_type_ids

    - input_ids: represent the token IDs of the input tokens after tokenization. Each token in the input text is converted into a numerical ID based on the tokenizer's vocabulary.

    - attention_mask: indicates which tokens should be attended to during processing. Binary mask where each position corresponds to a token in the input.

    - token_type_ids: represent the segment IDs for token types in the context of sequence pairs.

    2. Image input requires three pieces: visual_embeds, visual_token_type_ids, visual_attention_mask

    - visual_embeds: represents the embeddings of visual features in the model.

    - visual_token_type_ids: represent the token type IDs for visual toekens. In multi-modal transformers distinguishes between tokens representing visual features and tokens representing text

    - vision_attention_mask: guide the attention mechanism for visual tokens. Specifies which tokens should be attended to during processing.

### Create Dataset Class

In [ ]:
train_dataset['text'].to_list()

In [ ]:
input_text = train_dataset.loc[0,'text']
input_text_tokenized = tokenizer(train_dataset['text'].to_list(), return_tensors='pt', padding='max_length', max_length=512, truncation=True)

In [ ]:
len(input_text_tokenized['input_ids'])

In [ ]:
np.array(train_dataset['img'])

In [ ]:
from transformers import AutoImageProcessor, Swinv2Model
import torch
from datasets import load_dataset
import numpy as np
from PIL import Image

image = Image.open(os.path.join(data_directory,train_dataset.loc[0,'img']))
image_array = np.array(image)
tensor = torch.tensor(image_array)


image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")
viz_model = Swinv2Model.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")

inputs = image_processor(tensor, return_tensors="pt")

with torch.no_grad():
    outputs = viz_model(**inputs)

visual_embeds = outputs.last_hidden_state
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)

In [ ]:

import torch 
from transformers import BertTokenizer, VisualBertModel
from transformers import Swinv2Model,AutoImageProcessor
import os 
import pandas as pd
import jsonlines
from PIL import Image
import numpy as np


parent_directory = os.path.dirname(os.path.dirname(os.getcwd()))
data_directory = os.path.join(parent_directory,'data')


class DatasetBuilder():
    def __init__(self,model = None,tokenizer = None,json_file = None):
        self.parent_directory = os.path.dirname(os.path.dirname(os.getcwd()))
        self.data_directory = os.path.join(parent_directory,'data')
        self.json_file = json_file
        
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer)
        self.image_processor = AutoImageProcessor.from_pretrained(model)
        self.viz_model = Swinv2Model.from_pretrained(model)
        self.dataset = self.create_dataframe()

    def create_dataframe(self):
        dataset = pd.DataFrame(columns=['id','img','label','text'])
        with jsonlines.open(os.path.join(data_directory,self.json_file)) as reader:
            for data in reader:
                data_df = pd.DataFrame([data])
                dataset = pd.concat([dataset,data_df]).reset_index(drop=True)

        dataset['img'] = (data_directory+'/'+dataset['img']).apply(self.load_image)
        return dataset
    

    def tokenize_data(self,value):
        input = self.tokenizer(value['text'], return_tensors='pt', padding='max_length', max_length=512, truncation=True)
        target = torch.tensor(value['label']).type(torch.int64)
        
        try:
            image = self.image_processor(tensor, return_tensors="pt")
            
            
            outputs = self.viz_model(**image)

            visual_embeds = outputs.last_hidden_state

        except:
            visual_embeds = np.zeros(shape=(197, 768), dtype=float)

            
        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
        visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
        input.update(
            {
                "visual_embeds": visual_embeds,
                "visual_token_type_ids": visual_token_type_ids,
                "visual_attention_mask": visual_attention_mask,
                "label":target,
            }
        )

        return input
    
    def get_dataset(self):
        return self.dataset
  
    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset.loc[index])
        
        for k in inputs.keys():
            print(k, inputs[k].shape, inputs[k].dtype)

        return inputs

    def __len__(self):
        return len(self.dataset)

    def load_image(self,filepath):
        image = Image.open(filepath)
        image_array = np.array(image)
        return image_array

In [ ]:
tokenizer = 'google-bert/bert-base-uncased'
model = "microsoft/swinv2-base-patch4-window12-192-22k"
dataset = DatasetBuilder(model,tokenizer,'train.jsonl')

In [ ]:
dataset[0]